# **Despliege del Modelo con MLFlow**
---

En este notebook se desplegará el modelo utilizando la API de `mlflow` a través de la librería `requests`.

Comenzamos configurando el servidor de `mlflow` e importando las librerías necesarias:

In [ ]:
!pip install mlflow requests

In [ ]:
import mlflow
import os
import logging
import pandas as pd
from IPython.display import display

Adicionalmente, utilizaremos un servidor de `mlflow`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Utilizaremos `ngrok` para acceder al tablero de `mlflow`:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "2pEXurcBWiJur3b8zAfzNp2YbbE_82H8edeFMVSNbouejjwva" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Vamos a crear un experimento en MLFlow para este conjunto de datos:

In [ ]:
exp_id = mlflow.create_experiment(name="DiaBoost", artifact_location="mlruns/")

## **1. Carga de Datos**
---

Este conjunto de datos es una colección de datos médicos y demográficos de pacientes, junto con una etiqueta que indica si el paciente tiene o no diabetes. Los datos incluyen características como la edad, el sexo, el índice de masa corporal (IMC), la hipertensión, las cardiopatías, el historial de tabaquismo, el nivel de HbA1c y el nivel de glucosa en sangre.

El entrenamiento de un modelo de Machine Learning con estos datos puede ser útil para que los profesionales sanitarios identifiquen a los pacientes que pueden estar en riesgo de desarrollar diabetes y desarrollen planes de tratamiento personalizados. Además, el conjunto de datos puede ser utilizado por los investigadores para explorar las relaciones entre diversos factores médicos y demográficos y la probabilidad de desarrollar diabetes.

Estos datos provienen originalmente de [Electronic Health Records (EHRs)](https://www.cms.gov/priorities/key-initiatives/e-health/records); de allí fueron tomados, agrupados, procesados y republicados en [Kaggle](https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset).

Vamos a cargar este conjunto de datos:

In [ ]:
def create_logger():
  logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levenname)s - %(message)s')
  logger = logging.getLogger('Logger')
  logger.info('Logger creado')
  return logger


def download_firebase(url, logger):
  logger.info("Extrayendo el archivo desde Firebase")
  df = None
  try:
    df = pd.read_csv(url)
    logger.info("Archivo cargado")
  except requests.exceptions.RequestException as e:
    logger.info(f"Error al descargar el archivo CSV: {e}")
  except pd.errors.EmptyDataError:
    logger.info("El archivo CSV está vacío.")
  except Exception as e:
    logger.info(f"Ocurrió un error inesperado: {e}")
  return df

In [ ]:
# Cargar DataSet
url = 'https://firebasestorage.googleapis.com/v0/b/personalwp-8822c.appspot.com/o/diabetes_prediction_dataset.csv?alt=media&token=4d70d154-c3d0-4fa0-a3aa-9b9972dd3b95'
logger = create_logger()
data = download_firebase(url,logger)
display(data.head())

Este conjunto de datos tiene los siguientes campos:

| **Variable** | **Descripción** | **Tipo de dato** | **Rango/Valores posibles** |
|---|---|---|---|
| **gender** | Género del paciente | Categórico (string) | Female, Male, Other |
| **age** | Edad del paciente | Numérico (float) | 102 posibles valores entre 0.08 y 80.0 |
| **hypertension** | Indica si el paciente tiene hipertensión | Categórico (bool) | 0, 1 (0: No, 1: Sí) |
| **heart_disease** | Indica si el paciente tiene una enfermedad cardíaca | Categórico (bool) | 0, 1 (0: No, 1: Sí) |
| **smoking_history** | Antecedentes de tabaquismo del paciente | Categórico (string) | never, No Info, current, former, ever, not current |
| **bmi** | Índice de masa corporal del paciente | Numérico (float) | 4247 posibles valores entre 10.01 y 95.69 |
| **HbA1c_level** | Nivel promedio de glucosa en sangre del paciente (últimos meses) | Numérico (float) | 18 posibles valores entre 3.5 y 9.0 |
| **blood_glucose_level** | Nivel de glucosa en sangre del paciente (en el momento de la prueba) | Numérico (int) | 18 posibles valores entre 80 y 300 |
| **diabetes** | Indica si el paciente tiene diabetes | Categórico (bool) | 0, 1 (0: No, 1: Sí) |

Vamos a preprocesar los datos:

In [ ]:
# Valores atípicos en 'bmi'
seventy_fifth = data['bmi'].quantile(0.75)
twenty_fifth = data['bmi'].quantile(0.25)
iqr = seventy_fifth - twenty_fifth
upper = seventy_fifth + (10 * iqr)
outliers_bmi_upper = data[(data['bmi'] > upper)]

In [ ]:
# Eliminación de valores atípicos en 'bmi'
data = pd.merge(data, outliers_bmi_upper, indicator = True, how = 'outer').query('_merge == "left_only"').drop('_merge', axis = 1)

In [ ]:
# Eliminación de valores duplicados
data = data.drop_duplicates(keep = "first")

In [ ]:
# Variables Categóricas a Numéricas
data['gender'] = pd.factorize(data['gender'])[0]
data['smoking_history'] = pd.factorize(data['smoking_history'])[0]

In [ ]:
# Información del DataSet
data.info()

## **2. Modelamiento**
---

Ahora, veamos el entrenamiento de un modelo de `xgboost`, para escoger los mejores hiperparámetros instalamos `optuna`:

In [ ]:
!pip install optuna

In [ ]:
from xgboost import XGBClassifier

Dividimos el conjunto de datos en entrenamiento y prueba para validar la generalización del modelo:

In [ ]:
# Separación de la 'Data' (Características)
X = data.drop(columns = 'diabetes')
X.shape

In [ ]:
# Separación del 'Target' (Variable objetivo)
y = data['diabetes']
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
# Partición de los datos: 70% para entrenamiento, 30% para prueba y estratificación en las etiquetas (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
# Validación de la partición de los datos
print(f'Número de muestras en entrenamiento: {X_train.shape[0]}')
print(f'Número de muestras en prueba: {X_test.shape[0]}')
print(f'Número de características: {X_train.shape[1]}')

In [ ]:
import numpy as np
# Distribución de la variable objetivo en los conjuntos de entrenamiento y prueba
print(f'Distribución de clases en entrenamiento: {np.bincount(y_train)}')
print(f'Distribución de clases en prueba: {np.bincount(y_test)}')

Entrenamos el modelo:

In [ ]:
from sklearn.metrics import accuracy_score

def objective(trial):
  max_depth = trial.suggest_int("max_depth", 1, 100)
  n_estimators = trial.suggest_int("n_estimators", 1, 200)
  learning_rate = trial.suggest_float("learning_rate", 1e-6, 1, log = True)
  model = XGBClassifier(max_depth = max_depth, n_estimators = n_estimators, learning_rate = learning_rate)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  score = accuracy_score(y_test, y_pred)
  return score

Sobre este modelo, debe generar una versión con el
nombre `DiaBoost`.

In [ ]:
import optuna
study = optuna.create_study(direction = "maximize", storage = "sqlite:///hp.db", study_name = "DiaBoost")
study.optimize(func = objective, n_trials = 100, n_jobs = -1)

In [ ]:
# Mejor hiperparámetro
params = study.best_params
print(params)

In [ ]:
# Mejor metrica
score = study.best_value
print(score)

## **3. Despliegue** [1]
---

`mlflow` nos permite desplegar modelos como **REST APIs** de forma muy sencilla. Un REST API (acrónimo en inglés de *Representational State Transfer Application Programming Interface*) es un tipo de API (*Application Programming Interface*) que utiliza la arquitectura REST para proporcionar servicios web. REST es un conjunto de principios y restricciones que se utilizan para crear servicios web escalables y flexibles que pueden ser accedidos desde cualquier dispositivo o plataforma que tenga conexión a Internet.

<img src="https://drive.google.com/uc?export=view&id=1zNq0W7kTnw4nCN2hNGEevWXNfa6TKH7E" width="80%">

En un REST API, los datos son transferidos entre el cliente y el servidor a través de solicitudes HTTP estándar, como GET, POST, PUT y DELETE. Estas solicitudes se utilizan para realizar operaciones en los recursos que se encuentran en el servidor. Los recursos se identifican mediante URLs y los datos se transfieren en un formato estandarizado, como JSON o XML.

El uso de REST API se ha vuelto muy popular en los últimos años debido a que es un enfoque muy flexible y escalable para construir servicios web. Muchas aplicaciones móviles y web utilizan REST API para acceder a datos y realizar operaciones en ellos.

`mlflow` permite desplegar modelos que ya se encuentran en el registro por medio de un REST API sencillo que toma como entrada los datos de un modelo y devuelve las predicciones:

<img src="https://drive.google.com/uc?export=view&id=1glFzD_ngp-QMN8NWfQfjM3sUWJZnvdLY" width="80%">

Para crear el API de `mlflow` debemos especificar la url del servidor de seguimiento de `mlflow`:

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

Ahora, lanzamos el API con `mlflow`:

In [ ]:
command = """
mlflow models serve -m 'models:/DiaBoost/1' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

Esto genera un API que está ejecutándose en el puerto `8001`. Veamos cómo podemos enviarle datos con la librería `requests`:

In [ ]:
import requests

Vamos a enviarle dos registros del conjunto de test:

In [ ]:
data_request = X_test[:2].values.tolist()
display(data_request)

Finalmente, enviamos los datos para que el modelo desplegado nos de una predicción:

In [ ]:
r = requests.post("http://localhost:8001/invocations", json={"inputs": data_request})
print(r.text)

Como podemos ver, el API nos retorna las predicciones del modelo de una forma muy sencilla. Así mismo, `mlflow` nos permite hacer despliegues como aplicaciones web con un único comando.

## **4. Referencias**
---

**[1]** J. E. Camargo, J. S. Lara, E. Hernández, R. A. Superlano, and M. A. Rodríguez, "Despliege de Modelos con MLFlow," in Machine Learning and Data Science Specialization, Universidad Nacional de Colombia, 2023. [Online]. Available: https://colab.research.google.com/github/mindlab-unal/MLDS6/blob/main/u4/Despliegue_de_Modelos_con_MLFlow.ipynb
